# Importing the Dependencies

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import re

In [2]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer
port_stem = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Data Collection & Pre-Processing

In [3]:
raw_mail_data = pd.read_csv('mail_data.csv')

In [4]:
print(raw_mail_data)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [5]:
raw_mail_data.isnull().sum()

Category    0
Message     0
dtype: int64

In [6]:
mail_data = raw_mail_data.dropna(subset = "Message")

In [7]:
mail_data.head()
mail_data.shape

(5572, 2)

In [8]:
port_stem = PorterStemmer()

In [9]:
def generalise(text):
  text = text.lower()
  text = re.sub('[^a-z]',' ',text)
  text = text.split()
  text = [port_stem.stem(word) for word in text if not word in stopwords.words('english')]
  text = ' '.join(text)
  return text

In [10]:
mail_data['Message'] = mail_data['Message'].apply(generalise)

# Label Encoding

In [11]:
mail_data.loc[mail_data['Category'] == 'spam', 'Category',] = 0
mail_data.loc[mail_data['Category'] == 'ham', 'Category',] = 1

* spam - 0
 
* ham - 1

In [12]:
X = mail_data['Message']
Y = mail_data['Category']

In [13]:
print(X)

0       go jurong point crazi avail bugi n great world...
1                                   ok lar joke wif u oni
2       free entri wkli comp win fa cup final tkt st m...
3                     u dun say earli hor u c alreadi say
4                    nah think goe usf live around though
                              ...                        
5567    nd time tri contact u u pound prize claim easi...
5568                                b go esplanad fr home
5569                                    piti mood suggest
5570    guy bitch act like interest buy someth els nex...
5571                                       rofl true name
Name: Message, Length: 5572, dtype: object


In [14]:
print(Y)

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: object


# Splitting the data into training data & test data

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [16]:
print(X.shape)
print(X_train.shape)
print(X_test.shape)

(5572,)
(4457,)
(1115,)


Feature Extraction

In [17]:
feature_extraction = TfidfVectorizer()

X_train_features = feature_extraction.fit_transform(X_train)
X_test_features = feature_extraction.transform(X_test)

Y_train = Y_train.astype('int')
Y_test = Y_test.astype('int')

In [18]:
print(X_train)

1978    repli win weekli fifa world cup held send stop...
3989    hello sort town alreadi dont rush home eat nac...
3935                        come guoyang go n tell u told
4078    hey sathya till dint meet even singl time saw ...
4086    orang bring rington time chart hero free hit w...
                              ...                        
3772    hi wlcome back wonder got eaten lion someth no...
5191                                     sorri call later
5226                   prabha soryda reali frm heart sori
5390                                 nt joke serious told
860                               say somebodi name tampa
Name: Message, Length: 4457, dtype: object


In [19]:
print(X_train_features)

  (0, 4213)	0.2589400650794878
  (0, 1461)	0.2667514611844667
  (0, 4586)	0.2262397093334804
  (0, 4196)	0.20849904995757204
  (0, 2114)	0.4063431025874305
  (0, 1082)	0.35834686867924903
  (0, 5452)	0.290509466585464
  (0, 1661)	0.42618427077303406
  (0, 5323)	0.30666962616533244
  (0, 5387)	0.25151153775945895
  (0, 3969)	0.22422566614651127
  (1, 1512)	0.38429196872653903
  (1, 2577)	0.1885666763934136
  (1, 2680)	0.23351114363990885
  (1, 3145)	0.40305641073460635
  (1, 1403)	0.2692934502523703
  (1, 2171)	0.2067728630399252
  (1, 4069)	0.3606515676125802
  (1, 1314)	0.21635656162945344
  (1, 147)	0.23986064804016122
  (1, 4980)	0.29002750049719406
  (1, 4461)	0.30682233426850475
  (1, 2118)	0.2680254583784647
  (2, 4934)	0.44750216941002907
  (2, 4784)	0.36516228999264344
  :	:
  (4452, 2713)	0.38582078634356226
  (4452, 4434)	0.2735032810472839
  (4452, 3270)	0.30423106087062834
  (4452, 5438)	0.30095930979445307
  (4452, 2133)	0.2426917777586048
  (4452, 1967)	0.2111941686759436

# Logistic Regression

In [20]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(X_train_features, Y_train)

LogisticRegression()

In [21]:
prediction_on_test_data = LR.predict(X_test_features)
accuracy_on_test_data = accuracy_score(Y_test, prediction_on_test_data)

In [22]:
print('Accuracy on test data : ', accuracy_on_test_data)

Accuracy on test data :  0.968609865470852


# Decision Tree Classifier

In [23]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(X_train_features,Y_train)

DecisionTreeClassifier()

In [24]:
Predict_DT = DT.predict(X_test_features)

In [25]:
DT_score = accuracy_score(Y_test,Predict_DT)
print(DT_score)

0.9659192825112107


# Random Forest Classifier

In [26]:
from sklearn.ensemble import RandomForestClassifier

RF = RandomForestClassifier()
RF.fit(X_train_features,Y_train)

RandomForestClassifier()

In [27]:
Predict_RF = RF.predict(X_test_features)

In [28]:
RF_score = accuracy_score(Y_test,Predict_RF)
print(RF_score)

0.9766816143497757


# Building a Predictive System

In [29]:
def predictions(n):
    if n==1:
      return 'Ham mail'
    elif n == 0:
      return 'Spam mail'


def manual_testing(text):
    input_mail = {'Message': [text]}
    df_testing = pd.DataFrame(input_mail)
    df_testing['Message'] = df_testing['Message'].apply(generalise)
    new_x_text = df_testing['Message']
    input_data_features = feature_extraction.transform(new_x_text)

    LR_predictions = LR.predict(input_data_features)
    DT_predictions = DT.predict(input_data_features)
    RF_predictions = RF.predict(input_data_features)
    print("Logistic Regression Prediction: ",predictions(LR_predictions[0]))
    print("Decision Tree Prediction: ",predictions(DT_predictions[0]))
    print("Random Forest Prediction: ",predictions(RF_predictions[0]))

In [30]:
input_mail = str(input())
manual_testing(input_mail)

 Hello


Logistic Regression Prediction:  Ham mail
Decision Tree Prediction:  Ham mail
Random Forest Prediction:  Ham mail
